<a href="https://colab.research.google.com/github/pavan-kalam/Hack-A-Roo-Project-Deep-Fake/blob/main/Hack_A_Roo(Deep_Fake).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install flask pyngrok

In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from PIL import Image
import io
import os
from flask import Flask, request, jsonify, render_template_string
from pyngrok import ngrok
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Define models directory
MODELS_DIR = '/content/drive/MyDrive/Hack-A-Roo/'
if not os.path.exists(MODELS_DIR):
    os.makedirs(MODELS_DIR)

In [4]:
# Flask app setup
app = Flask(__name__)

# HTML Template (same as before)
HTML_TEMPLATE = '''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Deepfake Detector</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            max-width: 800px;
            margin: 0 auto;
            padding: 20px;
            background-color: #f5f5f5;
        }
        .container {
            background-color: white;
            padding: 20px;
            border-radius: 8px;
            box-shadow: 0 2px 4px rgba(0,0,0,0.1);
        }
        h1 {
            color: #333;
            text-align: center;
        }
        form {
            margin-bottom: 20px;
        }
        select, input[type="file"] {
            width: 100%;
            padding: 8px;
            margin: 10px 0;
            border: 1px solid #ddd;
            border-radius: 4px;
        }
        input[type="submit"] {
            background-color: #4CAF50;
            color: white;
            padding: 10px 20px;
            border: none;
            border-radius: 4px;
            cursor: pointer;
            width: 100%;
        }
        input[type="submit"]:hover {
            background-color: #45a049;
        }
        #result {
            margin-top: 20px;
            padding: 15px;
            border-radius: 4px;
            text-align: center;
            font-weight: bold;
        }
        .real {
            background-color: #dff0d8;
            color: #3c763d;
        }
        .fake {
            background-color: #f2dede;
            color: #a94442;
        }
    </style>
</head>
<body>
    <div class="container">
        <h1>Deepfake Image Detector</h1>
        <form action="/" method="post" enctype="multipart/form-data">
            <label for="model">Select Model:</label>
            <select name="model" id="model">
                {% for model in models %}
                <option value="{{ model }}">{{ model }}</option>
                {% endfor %}
            </select>
            <input type="file" name="image" accept="image/*" required>
            <input type="submit" value="Detect">
        </form>
        {% if result %}
        <div id="result" class="{{ 'fake' if 'FAKE' in result else 'real' }}">
            {{ result }}
        </div>
        {% endif %}
    </div>
</body>
</html>
'''

def get_available_models():
    models = []
    for file in os.listdir(MODELS_DIR):
        if file.endswith('.keras'):
            models.append(file)
    return sorted(models)

def preprocess_image(image):
    if image.mode != 'RGB':
        image = image.convert('RGB')
    image = image.resize((128, 128))
    image_array = np.array(image)
    image_array = image_array.astype('float32') / 255.0
    image_array = np.expand_dims(image_array, axis=0)
    return image_array

def load_selected_model(model_name):
    model_path = os.path.join(MODELS_DIR, model_name)
    return keras.models.load_model(model_path)

def predict_image(model, image):
    processed_image = preprocess_image(image)
    prediction = model.predict(processed_image, verbose=0)[0][0]
    result = "FAKE" if prediction >= 0.5 else "REAL"
    confidence = float(prediction) if prediction >= 0.5 else float(1 - prediction)
    confidence = confidence * 100
    return result, confidence

@app.route('/', methods=['GET', 'POST'])
def index():
    result = ""
    if request.method == 'POST':
        try:
            model_name = request.form['model']
            image_file = request.files['image']

            if image_file:
                image = Image.open(image_file)
                model = load_selected_model(model_name)
                prediction, confidence = predict_image(model, image)
                result = f"Prediction: {prediction} (Confidence: {confidence:.2f}%)"
        except Exception as e:
            result = f"Error: {str(e)}"

    models = get_available_models()
    return render_template_string(HTML_TEMPLATE, models=models, result=result)

# Set up ngrok
ngrok.set_auth_token("2g1JdKt7wOkTkDKLQwdgBIBrSkf_7qXqeFnUFPT4Hqegq9uXc")  # Replace with your ngrok auth token
public_url = ngrok.connect(5000)
print(f" * ngrok tunnel URL: {public_url}")

# Run the app
if __name__ == '__main__':
    app.run()

 * ngrok tunnel URL: NgrokTunnel: "https://b22e-34-91-191-229.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [11/Nov/2024 18:06:00] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Nov/2024 18:06:19] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Nov/2024 18:08:13] "GET / HTTP/1.1" 200 -
